# Análise e Visualização de Resultados de Dinâmica Molecular

Este notebook demonstra como gerar gráficos e análises a partir dos arquivos de saída do GROMACS para a simulação de dinâmica molecular da lisozima.

## Importação de Bibliotecas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

# Configurações para melhor visualização
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

## Função para Ler Arquivos .xvg do GROMACS

In [ ]:
def read_xvg(filename):
    """
    Lê arquivos .xvg do GROMACS e retorna os dados como array numpy
    
    Parameters:
    filename (str): Caminho para o arquivo .xvg
    
    Returns:
    numpy.ndarray: Array com os dados do arquivo
    """
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
        
        # Remove linhas de comentário (começam com # ou @)
        data_lines = [line.strip() for line in lines 
                     if not line.startswith('#') and not line.startswith('@') and line.strip()]
        
        # Converte para numpy array
        data = np.array([line.split() for line in data_lines], dtype=float)
        return data
    
    except FileNotFoundError:
        print(f"Arquivo {filename} não encontrado. Gerando dados simulados para demonstração.")
        return generate_mock_data(filename)

def generate_mock_data(filename):
    """
    Gera dados simulados para demonstração quando os arquivos reais não estão disponíveis
    """
    np.random.seed(42)  # Para resultados reproduzíveis
    
    if 'rmsd' in filename:
        time = np.linspace(0, 10, 1000)  # 10 ns, 1000 pontos
        rmsd = 0.15 + 0.05 * np.random.random(1000) + 0.02 * np.sin(time * 2)
        return np.column_stack([time, rmsd])
    
    elif 'rmsf' in filename:
        residues = np.arange(1, 130)  # 129 resíduos da lisozima
        rmsf = 0.1 + 0.15 * np.random.random(129)
        # Simula loops mais flexíveis
        rmsf[10:15] *= 2  # Loop 1
        rmsf[45:50] *= 1.8  # Loop 2
        rmsf[80:85] *= 2.2  # Loop 3
        return np.column_stack([residues, rmsf])
    
    elif 'giracao' in filename or 'gyration' in filename:
        time = np.linspace(0, 10, 1000)
        gyration = 1.45 + 0.02 * np.random.random(1000) + 0.005 * np.sin(time * 3)
        return np.column_stack([time, gyration])
    
    elif 'hbond' in filename:
        time = np.linspace(0, 10, 1000)
        hbonds = 115 + 10 * np.random.random(1000) + 3 * np.sin(time * 4)
        return np.column_stack([time, hbonds])
    
    else:
        # Dados genéricos
        time = np.linspace(0, 10, 1000)
        values = np.random.random(1000)
        return np.column_stack([time, values])

## 1. Análise RMSD (Root-Mean-Square Deviation)

O RMSD mede o desvio médio da estrutura em relação à configuração inicial. É um indicador importante da estabilidade estrutural da proteína.

In [ ]:
# Carrega dados RMSD
rmsd_data = read_xvg('rmsd.xvg')
time_rmsd = rmsd_data[:, 0]
rmsd_values = rmsd_data[:, 1]

# Cria o gráfico
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(time_rmsd, rmsd_values, color='#2E8B57', linewidth=1.5, alpha=0.8)
ax.fill_between(time_rmsd, rmsd_values, alpha=0.3, color='#2E8B57')

# Adiciona linha da média
mean_rmsd = np.mean(rmsd_values)
ax.axhline(y=mean_rmsd, color='red', linestyle='--', linewidth=2, 
           label=f'Média: {mean_rmsd:.3f} nm')

ax.set_xlabel('Tempo (ns)', fontsize=14)
ax.set_ylabel('RMSD (nm)', fontsize=14)
ax.set_title('RMSD do Backbone da Lisozima', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=12)

# Adiciona estatísticas no gráfico
textstr = f'Desvio padrão: {np.std(rmsd_values):.3f} nm\nValor final: {rmsd_values[-1]:.3f} nm'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)

plt.tight_layout()
plt.show()

print(f"Análise RMSD:")
print(f"Valor médio: {mean_rmsd:.3f} ± {np.std(rmsd_values):.3f} nm")
print(f"Valor mínimo: {np.min(rmsd_values):.3f} nm")
print(f"Valor máximo: {np.max(rmsd_values):.3f} nm")
print(f"Valor final: {rmsd_values[-1]:.3f} nm")

## 2. Análise RMSF (Root-Mean-Square Fluctuation)

O RMSF mede a flexibilidade de cada resíduo da proteína. Valores altos indicam regiões mais móveis (loops), enquanto valores baixos indicam regiões mais rígidas (estruturas secundárias).

In [ ]:
# Carrega dados RMSF
rmsf_data = read_xvg('rmsf.xvg')
residues = rmsf_data[:, 0]
rmsf_values = rmsf_data[:, 1]

# Cria o gráfico
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Gráfico principal
bars = ax1.bar(residues, rmsf_values, width=0.8, color='#4169E1', alpha=0.7, edgecolor='black', linewidth=0.5)

# Destaca resíduos com alta flexibilidade (>0.25 nm)
high_flexibility = rmsf_values > 0.25
for i, (res, rmsf, high_flex) in enumerate(zip(residues, rmsf_values, high_flexibility)):
    if high_flex:
        bars[i].set_color('#FF6347')

ax1.axhline(y=np.mean(rmsf_values), color='green', linestyle='--', linewidth=2,
           label=f'Média: {np.mean(rmsf_values):.3f} nm')
ax1.axhline(y=0.25, color='red', linestyle=':', linewidth=2,
           label='Limite alta flexibilidade (0.25 nm)')

ax1.set_xlabel('Número do Resíduo', fontsize=14)
ax1.set_ylabel('RMSF (nm)', fontsize=14)
ax1.set_title('Flutuação por Resíduo (C-alpha)', fontsize=16, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=12)

# Histograma de distribuição
ax2.hist(rmsf_values, bins=20, color='#4169E1', alpha=0.7, edgecolor='black')
ax2.axvline(x=np.mean(rmsf_values), color='green', linestyle='--', linewidth=2,
           label=f'Média: {np.mean(rmsf_values):.3f} nm')
ax2.set_xlabel('RMSF (nm)', fontsize=14)
ax2.set_ylabel('Frequência', fontsize=14)
ax2.set_title('Distribuição de Flexibilidade dos Resíduos', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=12)

plt.tight_layout()
plt.show()

# Identifica resíduos mais flexíveis
flexible_residues = residues[rmsf_values > 0.25]
print(f"Análise RMSF:")
print(f"Flexibilidade média: {np.mean(rmsf_values):.3f} ± {np.std(rmsf_values):.3f} nm")
print(f"Resíduos mais flexíveis (>0.25 nm): {len(flexible_residues)} de {len(residues)}")
if len(flexible_residues) > 0:
    print(f"Resíduos flexíveis: {flexible_residues.astype(int)}")

## 3. Análise do Raio de Giração

O raio de giração mede a compactação da proteína. Variações significativas podem indicar mudanças conformacionais importantes.

In [ ]:
# Carrega dados do raio de giração
gyration_data = read_xvg('giracao.xvg')
time_gyration = gyration_data[:, 0]
gyration_values = gyration_data[:, 1]

# Cria o gráfico
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico temporal
ax1.plot(time_gyration, gyration_values, color='#FF8C00', linewidth=1.5)
ax1.fill_between(time_gyration, gyration_values, alpha=0.3, color='#FF8C00')

# Adiciona linha da média
mean_gyration = np.mean(gyration_values)
ax1.axhline(y=mean_gyration, color='blue', linestyle='--', linewidth=2,
           label=f'Média: {mean_gyration:.3f} nm')

# Banda de variação normal (±1 desvio padrão)
std_gyration = np.std(gyration_values)
ax1.fill_between(time_gyration, 
                mean_gyration - std_gyration, 
                mean_gyration + std_gyration, 
                alpha=0.2, color='blue', label=f'±1σ ({std_gyration:.3f} nm)')

ax1.set_xlabel('Tempo (ns)', fontsize=14)
ax1.set_ylabel('Raio de Giração (nm)', fontsize=14)
ax1.set_title('Raio de Giração vs Tempo', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=12)

# Histograma de distribuição
ax2.hist(gyration_values, bins=30, color='#FF8C00', alpha=0.7, 
         edgecolor='black', density=True)

# Ajusta uma distribuição normal
mu, sigma = stats.norm.fit(gyration_values)
x = np.linspace(gyration_values.min(), gyration_values.max(), 100)
ax2.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, 
         label=f'Ajuste Normal\nμ={mu:.3f}, σ={sigma:.3f}')

ax2.axvline(x=mean_gyration, color='blue', linestyle='--', linewidth=2,
           label=f'Média: {mean_gyration:.3f} nm')

ax2.set_xlabel('Raio de Giração (nm)', fontsize=14)
ax2.set_ylabel('Densidade de Probabilidade', fontsize=14)
ax2.set_title('Distribuição do Raio de Giração', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=11)

plt.tight_layout()
plt.show()

# Análise de estabilidade
variation_percent = (std_gyration / mean_gyration) * 100
print(f"Análise do Raio de Giração:")
print(f"Valor médio: {mean_gyration:.3f} ± {std_gyration:.3f} nm")
print(f"Variação: {variation_percent:.2f}%")
print(f"Valor mínimo: {np.min(gyration_values):.3f} nm")
print(f"Valor máximo: {np.max(gyration_values):.3f} nm")

if variation_percent < 2:
    print("✓ Proteína muito estável (variação < 2%)")
elif variation_percent < 5:
    print("✓ Proteína estável (variação < 5%)")
else:
    print("⚠ Possível instabilidade estrutural (variação > 5%)")

## 4. Análise de Ligações de Hidrogênio

As ligações de hidrogênio são cruciais para a estabilidade estrutural das proteínas. Mudanças significativas podem indicar desnaturação ou mudanças conformacionais.

In [ ]:
# Carrega dados de ligações de hidrogênio
hbond_data = read_xvg('hbond_intra.xvg')
time_hbond = hbond_data[:, 0]
hbond_values = hbond_data[:, 1]

# Cria o gráfico
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Gráfico temporal
ax1.plot(time_hbond, hbond_values, color='#9932CC', linewidth=1.5, alpha=0.8)
ax1.fill_between(time_hbond, hbond_values, alpha=0.3, color='#9932CC')

# Adiciona linha da média
mean_hbond = np.mean(hbond_values)
ax1.axhline(y=mean_hbond, color='red', linestyle='--', linewidth=2,
           label=f'Média: {mean_hbond:.1f} ligações')

# Calcula média móvel para identificar tendências
window_size = 50
if len(hbond_values) > window_size:
    moving_avg = np.convolve(hbond_values, np.ones(window_size)/window_size, mode='valid')
    time_ma = time_hbond[window_size-1:]
    ax1.plot(time_ma, moving_avg, color='black', linewidth=2, 
             label=f'Média móvel (janela: {window_size})')

ax1.set_xlabel('Tempo (ns)', fontsize=14)
ax1.set_ylabel('Número de Ligações H', fontsize=14)
ax1.set_title('Ligações de Hidrogênio Intramoleculares vs Tempo', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=12)

# Análise de distribuição e autocorrelação
ax2.hist(hbond_values, bins=25, color='#9932CC', alpha=0.7, 
         edgecolor='black', density=True)

# Ajusta distribuição normal
mu_hb, sigma_hb = stats.norm.fit(hbond_values)
x_hb = np.linspace(hbond_values.min(), hbond_values.max(), 100)
ax2.plot(x_hb, stats.norm.pdf(x_hb, mu_hb, sigma_hb), 'r-', linewidth=2,
         label=f'Ajuste Normal\nμ={mu_hb:.1f}, σ={sigma_hb:.1f}')

ax2.axvline(x=mean_hbond, color='red', linestyle='--', linewidth=2,
           label=f'Média: {mean_hbond:.1f}')

ax2.set_xlabel('Número de Ligações H', fontsize=14)
ax2.set_ylabel('Densidade de Probabilidade', fontsize=14)
ax2.set_title('Distribuição de Ligações de Hidrogênio', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=12)

plt.tight_layout()
plt.show()

print(f"Análise de Ligações de Hidrogênio:")
print(f"Número médio: {mean_hbond:.1f} ± {np.std(hbond_values):.1f} ligações")
print(f"Valor mínimo: {np.min(hbond_values):.0f} ligações")
print(f"Valor máximo: {np.max(hbond_values):.0f} ligações")
print(f"Coeficiente de variação: {(np.std(hbond_values)/mean_hbond)*100:.1f}%")

## 5. Análise Combinada e Correlações

Análise das correlações entre diferentes propriedades estruturais para identificar padrões e comportamentos cooperativos.

In [ ]:
# Cria um gráfico combinado de todas as análises
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análise Completa de Dinâmica Molecular - Lisozima', fontsize=18, fontweight='bold')

# 1. RMSD
axes[0,0].plot(time_rmsd, rmsd_values, color='#2E8B57', linewidth=1.5)
axes[0,0].axhline(y=np.mean(rmsd_values), color='red', linestyle='--', alpha=0.8)
axes[0,0].set_xlabel('Tempo (ns)')
axes[0,0].set_ylabel('RMSD (nm)')
axes[0,0].set_title('RMSD do Backbone')
axes[0,0].grid(True, alpha=0.3)

# 2. RMSF
bars = axes[0,1].bar(residues, rmsf_values, width=0.8, color='#4169E1', alpha=0.7)
# Destaca resíduos flexíveis
for i, rmsf in enumerate(rmsf_values):
    if rmsf > 0.25:
        bars[i].set_color('#FF6347')
axes[0,1].axhline(y=np.mean(rmsf_values), color='green', linestyle='--', alpha=0.8)
axes[0,1].set_xlabel('Número do Resíduo')
axes[0,1].set_ylabel('RMSF (nm)')
axes[0,1].set_title('Flutuação por Resíduo')
axes[0,1].grid(True, alpha=0.3)

# 3. Raio de Giração
axes[1,0].plot(time_gyration, gyration_values, color='#FF8C00', linewidth=1.5)
axes[1,0].axhline(y=np.mean(gyration_values), color='blue', linestyle='--', alpha=0.8)
axes[1,0].set_xlabel('Tempo (ns)')
axes[1,0].set_ylabel('Raio de Giração (nm)')
axes[1,0].set_title('Raio de Giração')
axes[1,0].grid(True, alpha=0.3)

# 4. Ligações de Hidrogênio
axes[1,1].plot(time_hbond, hbond_values, color='#9932CC', linewidth=1.5)
axes[1,1].axhline(y=np.mean(hbond_values), color='red', linestyle='--', alpha=0.8)
axes[1,1].set_xlabel('Tempo (ns)')
axes[1,1].set_ylabel('Número de Ligações H')
axes[1,1].set_title('Ligações de Hidrogênio')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('analise_md_completa.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Matriz de Correlação

In [ ]:
# Interpola dados para ter o mesmo tamanho temporal
min_length = min(len(time_rmsd), len(time_gyration), len(time_hbond))
indices = np.linspace(0, min_length-1, min_length, dtype=int)

# Cria DataFrame para análise de correlação
df_corr = pd.DataFrame({
    'RMSD': rmsd_values[indices],
    'Raio_Giracao': gyration_values[indices],
    'Ligacoes_H': hbond_values[indices]
})

# Calcula matriz de correlação
correlation_matrix = df_corr.corr()

# Visualiza matriz de correlação
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Matriz de Correlação entre Propriedades Estruturais', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Matriz de Correlação:")
print(correlation_matrix.round(3))

## 7. Resumo Estatístico Final

In [ ]:
# Cria um resumo estatístico completo
print("=" * 60)
print("RESUMO ESTATÍSTICO DA SIMULAÇÃO DE DINÂMICA MOLECULAR")
print("=" * 60)

print(f"\n🔹 RMSD (Root-Mean-Square Deviation):")
print(f"   Média: {np.mean(rmsd_values):.3f} ± {np.std(rmsd_values):.3f} nm")
print(f"   Intervalo: [{np.min(rmsd_values):.3f} - {np.max(rmsd_values):.3f}] nm")
print(f"   Valor final: {rmsd_values[-1]:.3f} nm")

print(f"\n🔹 RMSF (Root-Mean-Square Fluctuation):")
print(f"   Média: {np.mean(rmsf_values):.3f} ± {np.std(rmsf_values):.3f} nm")
print(f"   Resíduos flexíveis (>0.25 nm): {len(rmsf_values[rmsf_values > 0.25])} de {len(rmsf_values)}")
print(f"   Resíduo mais flexível: {int(residues[np.argmax(rmsf_values)])} (RMSF: {np.max(rmsf_values):.3f} nm)")

print(f"\n🔹 Raio de Giração:")
print(f"   Média: {np.mean(gyration_values):.3f} ± {np.std(gyration_values):.3f} nm")
print(f"   Variação: {(np.std(gyration_values)/np.mean(gyration_values))*100:.2f}%")
print(f"   Intervalo: [{np.min(gyration_values):.3f} - {np.max(gyration_values):.3f}] nm")

print(f"\n🔹 Ligações de Hidrogênio:")
print(f"   Média: {np.mean(hbond_values):.1f} ± {np.std(hbond_values):.1f} ligações")
print(f"   Intervalo: [{np.min(hbond_values):.0f} - {np.max(hbond_values):.0f}] ligações")
print(f"   Coef. de variação: {(np.std(hbond_values)/np.mean(hbond_values))*100:.1f}%")

print(f"\n🔹 Avaliação de Estabilidade:")
stability_score = 0

# Critérios de estabilidade
if np.mean(rmsd_values) < 0.3:
    print(f"   ✓ RMSD baixo (<0.3 nm)")
    stability_score += 1
else:
    print(f"   ⚠ RMSD elevado (>0.3 nm)")

if (np.std(gyration_values)/np.mean(gyration_values))*100 < 5:
    print(f"   ✓ Raio de giração estável (variação <5%)")
    stability_score += 1
else:
    print(f"   ⚠ Raio de giração instável (variação >5%)")

if (np.std(hbond_values)/np.mean(hbond_values))*100 < 15:
    print(f"   ✓ Ligações H estáveis (variação <15%)")
    stability_score += 1
else:
    print(f"   ⚠ Ligações H instáveis (variação >15%)")

print(f"\n🎯 Score de Estabilidade: {stability_score}/3")
if stability_score == 3:
    print("   Proteína muito estável durante a simulação")
elif stability_score == 2:
    print("   Proteína moderadamente estável")
else:
    print("   Possível instabilidade estrutural")

print("=" * 60)